# 🧠 Github Copilot

- Copilot Agent - ATS
- Talk to the document; tldr
- Copilot Chats in Solution/Projects/Folders
    - Chat Histories
    - MCPs
    - Using Chat to fix Agent issue

<img src=images/llms-github-copilot.png>

# 💡 Boundaries and Encapsulation

Think clearly about modularity, leakage, and autonomy of parts—essential for managing complexity as systems grow

Most modern IDEs and languages tend to **default to "public everything"**, especially in auto-generated code, scaffolding, or templates

- **C# (Visual Studio)**
  - Class templates, scaffolds often generate `public class`, `public methods`, etc
- **Java (Eclipse, IntelliJ)**
  - Class wizards and method stubs tend to default to `public`
- **TypeScript / JavaScript (VS Code)**
  - Everything is effectively public unless you explicitly use `private`, `#` (in JS), or `export`/`not exported`
- **Python**
  - No real access modifiers; all attributes/methods are public unless prefixed by `_`, and even then it’s just convention
- **Go**
  - Identifier casing controls visibility (`UpperCase = exported/public`), so public APIs are the norm

  __Why its problem__
- Unrestricted visibility **destroys encapsulation**
- It **invites coupling** across bounded contexts, modules, or aggregates
- Makes code harder to change, test, or evolve—**opposite of "Designing for Change."**


# 🕸️ Distributed Systems

## 🕸️ Overview

- DDD helped us model complex business logic in a modular way
- As domains grow, they can't live in one place (codebase or server) forever
- We move from Modular Monoliths → Microservices → Distributed Systems

*If we break a system into Bounded Contexts, how do they talk to each other?* 👈

__What?__

A distributed system is a system whose components are located on different networked computers, which communicate and coordinate their actions by passing messages

Examples:
- Microservices Architecture
- Cloud-native apps (e.g. Netflix)
- Database clusters
- Event-driven systems

__When?__

When you NEED them:
- Scale: Users, data, compute
- Resilience: One node shouldn't bring the system down
- Bounded Contexts in DDD grow into deployable, independently scalable units

Trade-offs:
- Simplicity vs Flexibility
- Coordination vs Performance

<img src=images/distributed-systems.png width=800>

## 🕸️ Core Concepts

__Latency and Network__
- Calls are not local anymore
- Communication = over the network = slower, less reliable

__CAP Theorem__
- You can’t have all 3: Consistency, Availability, Partition Tolerance
- Pick your trade-offs (this is directly related to domain rules!)

__Failures are Normal__
- Nodes fail, messages get lost
- Systems must expect and handle failure

__Data Consistency__
- Eventual vs Strong
- Domain rules guide how much consistency you need

__Distributed Transactions?__
- Saga Patterns, Compensation, Idempotency — not simple DB transactions

<img src=images/cap-theorem.webp width=800>

## 🧩 Domain Driven Design

*DDD gave us the language to model domains — Distributed Systems are how we scale and deliver them in the real world*

- DDD leads to modular boundaries, which scale into distributed components
- Distributed Systems introduce latency, failures, and complexity
- We must design with network, consistency, and communication in mind
- The domain drives the architecture — even in distributed environments

<img src=images/eventual-consistency.png width=800>

## 📨 Queues

__Why__

- Decoupling
- Buffering
- Load leveling
- Async Processing

__Example Scenarios__

- Order Processing in e-commernce
- Background job scheduling
- Event-driven microservices

__Limitation of Redis / Importance of RabbitMQ / Kafka__

- Redis
- RabbitMQ
- Kafka
- IoT Messaging - Protocol Importance

### 📨 Pub/Sub vs Point-to-Point

__Point-to-Point (Queue-based Messaging)__
- Model: Producer → Queue → One Consumer
- A producer sends messages to a queue
- Only one consumer processes each message
- Once the message is consumed, it's gone from the queue
- Use Case: Task processing, work distribution, load balancing
- Example Scenarios:
    - A web app sends an image for processing
    - Message goes to a queue
    - One worker picks it up and processes it

__Publish/Subscribe Messaging__
- Model: Publisher → Topic → Multiple Subscribers
- A publisher sends messages to a topic
- All active subscribers to that topic receive a copy of the message
- Use Case: Event notification, broadcasting information
- Example Scenario:
    - A service publishes a “user signed up” event
    - Billing, Email, and Analytics services each receive the event and act on it

### 📨 Atleast-once vs Exactly-once

__At-Least-Once Delivery__
- What it means: A message is delivered one or more times
- Risk: Duplicates may occur
- Why: The system retries sending if it doesn’t get an acknowledgment
- Your responsibility: Ensure your consumer is idempotent (can safely handle duplicates)
- Example Use Case: Sending an email—better to send twice than to not send at all

__Exactly-Once Delivery__
- What it means: A message is delivered only once, no duplicates, no losses
- Harder to achieve: Requires coordination between sender, broker, and consumer
- Often involves:
    - Transactions
    - Deduplication logic
    - Message offsets tracking
- Example Use Case: Processing a bank transaction—duplicates would be unacceptable

## 📨 Messaging Systems

### 📨 Traditional Systems

- Queues in Databases

In [ ]:
-- Enable Service Broker
ALTER DATABASE [YourDatabaseName] SET ENABLE_BROKER;

-- Create message type
CREATE MESSAGE TYPE SimpleMessage VALIDATION = NONE;

-- Create contract
CREATE CONTRACT SimpleContract (SimpleMessage SENT BY INITIATOR);

-- Create queue
CREATE QUEUE SimpleQueue;

-- Create service
CREATE SERVICE SimpleService ON QUEUE SimpleQueue (SimpleContract);

In [ ]:
using System.Data.SqlClient;

var connectionString = "Server=localhost;Database=YourDatabaseName;Integrated Security=true";

using var connection = new SqlConnection(connectionString);
connection.Open();

using var sendCmd = connection.CreateCommand();
sendCmd.CommandText = @"
        DECLARE @DialogHandle UNIQUEIDENTIFIER;
        BEGIN DIALOG CONVERSATION @DialogHandle
            FROM SERVICE [SimpleService]
            TO SERVICE 'SimpleService'
            ON CONTRACT [SimpleContract]
            WITH ENCRYPTION = OFF;

        SEND ON CONVERSATION @DialogHandle
        MESSAGE TYPE [SimpleMessage]
        ('Hello from SQL Server Queue');
    ";
sendCmd.ExecuteNonQuery();

In [ ]:
using var receiveCmd = connection.CreateCommand();
receiveCmd.CommandText = @"
    WAITFOR (
        RECEIVE TOP(1) CONVERT(VARCHAR(MAX), message_body) AS Message
        FROM SimpleQueue
    ), TIMEOUT 5000;
";
var message = receiveCmd.ExecuteScalar();
Console.WriteLine($"Received: {message}");

### ⚡ Message Brokers

- Redis
- __Message Brokers__
    - RabbitMQ
    - Kafka
    - MQTT

<img src=images/rabbitmq.png>

- https://www.rabbitmq.com/tutorials

<img src=images/kafka.png>

- https://kafka.apache.org/intro

<img src=images/kafka-streams.png height=400>

- The story of too much data and why we ended up writing our own "Snarter MQTT Server" 👈
    - https://github.com/dotnet/MQTTnet
    - https://github.com/microsoft/Trill 🥰

### 🗄️ PostgreSQL

In [ ]:
docker run --name pgmq -e POSTGRES_PASSWORD=pgmqpass -p 5432:5432 -d quay.io/tembo/pgmq-pg:latest

In [ ]:
-- Connect to your db (e.g., psql or any SQL tool)

-- Create the extension
CREATE EXTENSION IF NOT EXISTS pgmq;

-- Create a queue
SELECT * FROM pgmq_create('my_queue');

-- Send a message
SELECT * FROM pgmq_send('my_queue', 'Hello from pgmq!');

-- Read a message (doesn't delete it)
SELECT * FROM pgmq_read('my_queue', 30, 1);

-- Pop (read and delete) a message
SELECT * FROM pgmq_pop('my_queue', 30);

In [ ]:
#r "nuget: Npgsql"

In [ ]:
using Npgsql;

const string ConnStr = "Host=localhost;Username=postgres;Password=pgmqpass;Database=eventum";

static void SendMessage(NpgsqlConnection conn, string message)
{
    using var cmd = new NpgsqlCommand("SELECT * FROM pgmq_send('my_queue', @msg)", conn);
    cmd.Parameters.AddWithValue("msg", message);
    cmd.ExecuteNonQuery();
}

static void ReceiveMessage(NpgsqlConnection conn)
{
    using var cmd = new NpgsqlCommand("SELECT * FROM pgmq_pop('my_queue', 30)", conn);
    using var reader = cmd.ExecuteReader();
    while (reader.Read())
    {
        var body = reader["message"].ToString();
        Console.WriteLine(body);
    }
}

using var conn = new NpgsqlConnection(ConnStr);
conn.Open();

SendMessage(conn, "Hello from C# via pgmq");
ReceiveMessage(conn);

### 🗄️ Vanilla Database

In [ ]:
CREATE TABLE messages (
    id SERIAL PRIMARY KEY,
    content TEXT
);

In [ ]:
using Npgsql;

const string ConnectionString = "Host=localhost;Username=postgres;Password=yourpassword;Database=eventum";

using var conn = new NpgsqlConnection(ConnectionString);
conn.Open();

// Send message
using (var cmd = new NpgsqlCommand("INSERT INTO messages (content) VALUES (@msg)", conn))
{
    cmd.Parameters.AddWithValue("msg", "Hello, PostgreSQL!");
    cmd.ExecuteNonQuery();
}

// Receive message
using (var cmd = new NpgsqlCommand("SELECT content FROM messages ORDER BY id DESC LIMIT 1", conn))
using (var reader = cmd.ExecuteReader())
{
    while (reader.Read())
    {
        Console.WriteLine(reader.GetString(0));
    }
}

### 🌐 Cloud

__AWS__
- https://aws.amazon.com/messaging

__Azure__
- https://azure.microsoft.com/en-us/solutions/messaging-services
- CosmosDB

__Google__
- Firebase

In [ ]:
// Sample Change Feed Processor setup (not full implementation)
ChangeFeedProcessor processor = container
    .GetChangeFeedProcessorBuilder<Message>("processorName", async (changes, token) =>
    {
        foreach (var message in changes)
        {
            Console.WriteLine($"Message: {message.content}");
            // Custom logic here
        }
    })
    .WithInstanceName("instance1")
    .WithLeaseContainer(leaseContainer)
    .Build();

await processor.StartAsync();

- https://learn.microsoft.com/en-us/azure/architecture/databases/guide/transactional-outbox-cosmos
    - https://github.com/AzureCosmosDB/quickstart-nosql-dotnet

In [ ]:
#r "nuget: Newtonsoft.Json"
#r "nuget: Microsoft.Azure.Cosmos"

In [ ]:
using Microsoft.Azure.Cosmos;

// first lets check if environment variable COSMOSDB is set
if (string.IsNullOrEmpty(Environment.GetEnvironmentVariable("COSMOSDB")))
{
    Console.WriteLine("Please set the COSMOSDB environment variable with your Cosmos DB connection string.");
    return;
}

var client = new CosmosClient(
    connectionString: Environment.GetEnvironmentVariable("COSMOSDB")
);
var database = await client.CreateDatabaseIfNotExistsAsync("trainings");
var container = client.GetContainer("trainings", "eventum");

In [ ]:
// List items
var query = container.GetItemQueryIterator<dynamic>("SELECT * FROM c");
while (query.HasMoreResults)
{
    var response = await query.ReadNextAsync();
    foreach (var item in response)
        Console.WriteLine($"Item: {item}");
}

In [ ]:
record Item(string Id, string partitionKey, string Content);

//Create item
var item = new Item("item1", "testing", "Hello, Cosmos DB!");
var createResponse = await container.UpsertItemAsync<Item>(item, new PartitionKey(item.partitionKey)); //CreateItemAsync(item);
Console.WriteLine($"Created item: {createResponse.Resource}");

// // Read item
// var itemResponse = await container.ReadItemAsync<Item>(
//     id: item.Id,
//     partitionKey: new PartitionKey("testing")
// );
// Console.WriteLine($"Read item: {itemResponse.Resource}");

In [ ]:
// Update item
itemResponse.Resource.content = "Updated content";
var updateResponse = await container.ReplaceItemAsync(itemResponse.Resource, itemId, partitionKey);
Console.WriteLine($"Updated item: {updateResponse.Resource}");
// Delete item
var deleteResponse = await container.DeleteItemAsync<dynamic>(itemId, partitionKey);
Console.WriteLine($"Deleted item: {deleteResponse.Resource}");

- https://cloudevents.io
    - https://github.com/cloudevents/sdk-csharp
- https://supabase.com

### 📨 Other Concerns

__Idempotency__
- Definition: An operation is idempotent if performing it multiple times has the same effect as doing it once
- Why it matters: In at-least-once delivery, messages may be retried → your code must not break or double-process
- Example
    - POST /ship-order/123 → Ship order #123
    - If the message is processed twice:
    - Without idempotency: the order might be shipped twice
    - With idempotency: the system detects it's already processed order #123 → does nothing
- Common approaches:
    - Deduplication keys
    - Storing processed message IDs
    - Using natural idempotent operations (e.g., setting a value)

__Message Ordering__
- Problem: Messages may arrive out of order due to retries, network delays, or partitioning
- Critical when: Actions depend on a sequence (e.g., “order placed” → “order paid” → “order shipped”)
- Challenges:
    - Pub/Sub systems (like Kafka) can maintain order within a partition, but not globally.
    - Point-to-point queues may preserve order if single consumer, but retries and concurrency can disrupt it.
- Solutions:
    - Sequence numbers in messages
    - Partitioning based on keys (e.g., Kafka with consistent hashing)
    - Processing through single-threaded consumers per entity (e.g., per order ID)

# 🧩 Domain Driven Design

## 🛠️ Business Logic & Infrastructure

__Business Logic__

This is the heart of your application — the rules and behavior that represent your domain, examples:
- Calculating a discount for a customer
- Validating that an order can be placed
- Changing the status of a payment
- Checking if a user has permission to perform an action

It belongs to:
- Domain Layer
- Application Layer (or orchestration logic)

__Infrastructure__

This is the technical implementation — the code that deals with external systems or frameworks, examples:
- Sending an email
- Saving to a database
- Calling an external API
- Writing to a log file

It belongs to:
- Infrastructure Layer (also called adapters or ports in Hexagonal/Clean Architecture)

__Why This Separation Matters__
- Maintainability: You can change infrastructure (e.g., move from SQL to MongoDB) without touching your core domain logic
- Testability: Business rules can be tested in isolation, without needing a database or network
- Clarity: Your domain code tells the story of the business, not how things are wired

In [ ]:
// ✅ the right place for eligibility check
class LoanApplication
{
    public decimal Income { get; }
    public int CreditScore { get; }

    public LoanApplication(decimal income, int creditScore)
    {
        Income = income;
        CreditScore = creditScore;
    }

    public bool IsEligible() => // why we are not naming it Validate() 👈
        Income > 50000 && CreditScore >= 700;
}

interface ILoanRepository { bool Save(LoanApplication loan); }
// ❌ not a place for eligibility check
class SqlLoanRepository : ILoanRepository
{
    public bool Save(LoanApplication loan) { /* save to SQL DB */return false; }
}

class LoanProcessor
{
    readonly ILoanRepository repository;

    public LoanProcessor(ILoanRepository repository) =>
        this.repository = repository;

    public bool Process(LoanApplication loan) =>
        loan.IsEligible() && this.repository.Save(loan);
}

__DDD Naming Guideline__

Use ubiquitous language — terms that the domain experts (business stakeholders) use, in most business domains, terms like "Processor" are technical jargon. A better name reflects what the domain is doing, not how the code processes it

🔁 Better DDD-style alternatives for LoanProcessor:
- LoanApprovalService, clearly reflects a business operation
- LoanApprovalUseCase, Good in application layer (use-case-oriented)
- LoanOfficer, Metaphor from the real world (person role)
- LoanEvaluator, Focuses on evaluating business criteria

__In CQRS (Command Query Responsibility Segregation):__
- Commands → Change state
- Queries → Read data
- Handlers → Execute intent

In [ ]:
record ApproveLoanCommand(decimal Income, int CreditScore); // but why we need this when we have LoanApplication

class ApproveLoanCommandHandler
{
    readonly ILoanRepository repository;

    public ApproveLoanCommandHandler(ILoanRepository repository) =>
        this.repository = repository;

    public bool Handle(ApproveLoanCommand command)
    {
        var loan = new LoanApplication(command.Income, command.CreditScore);
        return loan.IsEligible() && this.repository.Save(loan);
    }
}

## 🧺 Aggregates

### 🚀 Aggregate Design

A core tactical pattern in DDD is the Aggregate, which defines a transactional consistency boundary around a cluster of Entities and Value Objects
- The rule of Aggregates is that only a single instance should be modified within a single transaction.

When a command on one Aggregate instance requires business rules to execute on other Aggregates, DDD advocates for eventual consistency
- While changes within a single Aggregate are transactionally consistent, consistency between Aggregates is often achieved asynchronously over time

In [ ]:
// State Inside SalesOrder Aggregate
enum SalesOrderState { PreSale, Approved, Completed }

class SalesOrder
{
    public Guid Id { get; private set; }
    public SalesOrderState State { get; private set; }

    public SalesOrder Approve() { return null; }
    public SalesOrder Complete() { return null; }
}

In [ ]:
// SalesOrder + Separate WorkflowExecution Aggregate

class SalesOrder
{
    public Guid Id { get; private set; }
    public Guid WorkflowExecutionId { get; private set; }
}

enum WorkflowState { PreSale, Approved, Completed }
class WorkflowExecution
{
    public Guid Id { get; private set; }
    public Guid SalesOrderId { get; private set; }
    public WorkflowState State { get; private set; }

    public WorkflowExecution Approve() { return null; }
    public WorkflowExecution Complete() { return null; }
}

__Pros and Cons__

| Criteria                     | State in `SalesOrder`                               | Separate `WorkflowExecution`                                 |
|------------------------------|------------------------------------------------------|-------------------------------------------------------------|
| **Simplicity**               | ✅ Fewer objects, easier to reason about             | ❌ Adds indirection, more infrastructure                   |
| **Encapsulation**            | ❌ State logic clutters SalesOrder                   | ✅ State logic isolated and focused                        |
| **Scalability**              | ❌ Tight coupling limits scalability                 | ✅ Loosely coupled, better for distributed systems         |
| **Transactional Boundaries** | ❌ Hard to keep clean if other Aggregates involved   | ✅ Only one Aggregate modified per transaction             |
| **State Transitions**        | ❌ Risks mixing responsibilities                     | ✅ Clear modeling of long-running workflows                |
| **Reusability**              | ❌ Hard to apply to other domains                    | ✅ WorkflowExecution can be reused across contexts         |
| **Complexity**               | ✅ Lower for simple apps                             | ❌ Higher infrastructure and coordination overhead         |
| **Eventual Consistency**     | ❌ Hard to model async processes                     | ✅ Natural fit with message/event-based workflows          |

### 🚀 Eventual Consistency

- Replicated Data Systems
- Weak Guarantee about "when" replicas will converge
- Confusing behavior for application developers coming from "single-threaded" mindset
- Bugs can be subtle and challenging to detect
    - Network Interruptions
    - High Concurrency

### 🚀 Why ?

__Scalability and Performance__
- Embracing eventual consistency outside Aggregate boundaries allows systems to perform and scale better, especially in high-traffic or distributed environments
- Large-cluster Aggregates, which might be needed for immediate consistency, can limit performance and scalability

__Distributed Systems__
- DDD acknowledges that in distributed systems, particularly those spanning multiple bounded contexts or geographical locations, immediate consistency across all replicas is often impractical and costly
- Eventual consistency, often facilitated by Domain Events and Event-Driven Architectures, allows different Bounded Contexts to remain loosely coupled and operate autonomously

__Domain Experts' Perspective__
- Domain experts often find the concept of delayed consistency more acceptable than developers, as they are aware of real-world business delays that occur regardless of computer automation
- This alignment with business realities helps design more practical and effective systems

__Simpler Application Code (at higher level)__
- By having the database or underlying system provide stronger guarantees through transactions, the application code can be simpler
- However, when transactions are abandoned, the complexity shifts to the application layer to manage consistency
- DDD's approach with Aggregates and Domain Events aims to provide a structured way to handle this complexity

__Event Sourcing__
- Event Sourcing, a technique that stores all changes to application state as a log of immutable events, pairs well with eventual consistency within DDD
- By replaying these events, the state of an Aggregate can be reconstituted, and views can be derived, often asynchronously. This naturally leads to eventual consistency for derived views

## ➕ Specification Pattern

In [ ]:
record Employee(int ID, string Name, string Department,
    DateTime JoiningDate,
    bool IsActive, bool IsAssigned);

IEnumerable<Employee> table; // ..
var q = table.AsQueryable();

q = q.Where(e => e.IsActive);
q = q.Where(e => e.IsAssigned);

foreach(var r in q) { }

var result = q.ToArray();
foreach(var r in result.OrderBy(e => e.JoiningDate)) { }

- https://deviq.com/design-patterns/specification-pattern
- https://specification.ardalis.com

In [ ]:
// SolenLms Example
//// 01 / Course Management / Features / Instructors / Queries
////// Ardalis Specification